In [52]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
from tensorflow.keras import preprocessing , utils
from gensim.models import Word2Vec
import re

In [53]:
lines = open('data/movie_lines.txt',  encoding='utf-8', errors ='ignore').read().split('\n')
conversations_lines = open('data/movie_conversations.txt',  encoding='utf-8', errors ='ignore').read().split('\n')

In [54]:
lines[:5]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."]

In [55]:
conversations_lines[:5]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']"]

In [56]:
# create a dictionary with key = id and value = text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [57]:
# create a list of all conversations' lines
# lsit of lists
conversations = [ ]
for line in conversations_lines:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations.append(_line.split(','))

In [58]:
conversations[:5]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208']]

In [59]:
# sort the lines into inupt texts and output texts
inputs = []
outputs = []

for conversation in conversations:
    for i in range(len(conversation)-1):
        inputs.append(id2line[conversation[i]])
        outputs.append(id2line[conversation[i+1]])

In [60]:
# load a couple of sorted input-output texts
for i in range(5):
    print(inputs[i])
    print(outputs[i]+'\n')
    

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



In [61]:
print(len(inputs))
print(len(outputs))

221616
221616


In [62]:
# change word format and remove unnecessary characters
def clean_text(text):

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [63]:
# clean the data
clean_inputs = []
for text in inputs:
    clean_inputs.append(clean_text(text))
    
clean_outputs = []    
for text in outputs:
    clean_outputs.append(clean_text(text))

In [64]:
# load a couple of sorted input-output texts
for i in range(5):
    print(clean_inputs[i])
    print(clean_outputs[i]+'\n')

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please

not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night

you are asking me out  that is so cute that is your name again
forget it

no no it is my fault  we did not have a proper introduction 
cameron



In [65]:
# find the length of sentences
lengths = []
for text in clean_inputs:
    lengths.append(len(text.split()))
for text in clean_outputs:
    lengths.append(len(text.split()))

# create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [66]:
lengths.describe()

,counts
count,443232.000000
mean,10.872094
std,12.215895
min,0.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,555.000000


In [67]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
24.0
32.0
58.0


In [99]:
# remove input and output texts that are shorter than 2 words and longer than 8 words.
min_line_length = 2
max_line_length = 10

# filter out the input texts that are too short/long
short_inputs_temp = []
short_outputs_temp = []

i = 0
for text in clean_inputs:
    if len(text.split()) >= min_line_length and len(text.split()) <= max_line_length:
        short_inputs_temp.append(text)
        short_outputs_temp.append(clean_outputs[i])
    i += 1

# filter out the output texts that are too short/long
short_inputs = []
short_outputs = []

i = 0
for text in short_outputs_temp:
    if len(text.split()) >= min_line_length and len(text.split()) <= max_line_length:
        short_outputs.append(text)
        short_inputs.append(short_inputs_temp[i])
    i += 1

In [100]:
# Compare the number of lines we will use with the total number of lines.
print('# of questions:', len(short_inputs))
print('# of answers:', len(short_outputs))
print('% of data used: {}%'.format(round(len(short_inputs)/len(inputs),2)*100))

# of questions: 74473
# of answers: 74473
% of data used: 34.0%


In [101]:
# add the end of sentence token to the end of every output.
# add the go token to the beginning of every output.
for i in range(len(short_outputs)):
    short_outputs[i] = '<GO>'+short_outputs[i]+' <EOS>'

In [102]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(short_inputs + short_outputs)
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 27001


In [103]:
vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list, vocabulary

In [108]:
# encoder_input_data
tokenized_inputs = tokenizer.texts_to_sequences(short_inputs)
max_len = max([len(x) for x in tokenized_inputs])
padded_inputs = preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=max_len, padding='post')
encoder_input_data = np.array(padded_inputs, dtype='uint8')
print(encoder_input_data.shape, max_len)

(74473, 12) 12


In [109]:
# decoder_input_data
tokenized_outputs = tokenizer.texts_to_sequences(short_outputs)
max_len = max([len(x) for x in tokenized_outputs])
padded_outputs = preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=max_len, padding='post')
decoder_input_data = np.array(padded_outputs, dtype='uint8')
print(decoder_input_data.shape, max_len)

(74473, 13) 13


In [110]:
# decoder_output_data
tokenized_outputs = tokenizer.texts_to_sequences(short_outputs)
for i in range(len(tokenized_outputs)) :
    tokenized_outputs[i] = tokenized_outputs[i][1:-1]
padded_outputs = preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=max_len, padding='post')
onehot_outputs = utils.to_categorical(padded_outputs ,VOCAB_SIZE)
decoder_output_data = np.array(onehot_outputs, dtype='uint8')
print(decoder_output_data.shape)

MemoryError: Unable to allocate 97.4 GiB for an array with shape (968149, 27001) and data type float32

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=(max_line_length, ))
encoder_embedding = tf.keras.layers.Embedding(len(outputs_vocab_to_int), 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=(max_line_length,  ))
decoder_embedding = tf.keras.layers.Embedding(len(outputs_vocab_to_int), 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense(len(outputs_vocab_to_int), activation=tf.keras.activations.softmax ) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(),loss='categorical_crossentropy', metrics = ['accuracy'])

model.summary()

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=32, epochs=10 ) 
model.save( 'model.h5' ) 

In [ ]:
import matplotlib.pyplot as plt
accuracy = model.history.history['accuracy']
loss = model.history.history['loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'g')
plt.plot(epochs, loss, 'b')
plt.legend(['Accuracy','Loss'])
plt.show()

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model